In [1]:
# Step 1. Import all Packages: Pandas, Numpy, and Statistics
#
import pandas as pd
import numpy as np
import statistics as st
import scipy
#from scipy import stats
import scipy.optimize as spo
from scipy.optimize import minimize
from scipy.optimize import fsolve

In [2]:
# Step 2. User Inputs
#
InputPath="c:/Algo/"
OutputPath="c:/Algo/"

# MI Data Input File
MI_InputDataFile="MIData_KRG.csv"

# MI Output Datafile
MI_OutputDataFile="MIParameters.csv"



In [3]:
# Stepp 3. Read MI Datafile
#
MIData_df = pd.read_csv(InputPath+MI_InputDataFile)
#MIData_df = pd.read_csv(MIDataFile)

# Display first 5 rows
MIData_df.head()

,Date,Size,Volatility,POV,Cost
0,8/14/2020,0.0927,0.498829,0.0303,-0.357895
1,8/14/2020,0.0568,0.344376,0.0333,171.617139
2,8/14/2020,0.0534,0.193020,0.0415,-26.072693
3,8/14/2020,0.0580,0.471967,0.0478,260.844290
4,8/14/2020,0.1500,0.337130,0.0538,126.922787


In [4]:
# Step 4. Define Variables
#
Size = MIData_df['Size'].to_numpy()
Volatility = MIData_df['Volatility'].to_numpy()
POV = MIData_df['POV'].to_numpy()
Cost = MIData_df['Cost'].to_numpy()

# Number of data points
N=Cost.size
print(N)

print(Cost)

233135
[ -0.35789526 171.6171393  -26.0726929  ...  95.18516373  62.5457652
 154.009555  ]


In [5]:
# Step 5. Define Optimization Objective Function
#
# We want to estimate the parameters: a1, a2, a3, a4, and b1
# The x-input variables are Size, Volatility, and POV
# The y-input variables is Cost
# The optimization objective function uses the IStar Model
# f = sum (MI - Cost)^2

def optimize1(initial_guess, Size, Volatility, POV, Cost):
    a1, a2, a3, a4, b1 = initial_guess
    return np.sum(np.square((a1 * (Size ** a2) * (Volatility ** a3) * (b1 * (POV ** a4) + (1 - b1))) - Cost))


In [6]:
# Step 6. Run Non-Linear Optimization
#

# Start with initital guess for the parameter values
a1 = 1000
a2 = 0.5
a3 = 0.5
a4 = 0.5
b1 = 0.95
initial_guess = [a1, a2, a3, a4, b1]

# a1, a2, a3, a4 > 0
# 0 <= b1 <= 1

# Set Upper and Lower Bounds on Constraints
LB = [0.0001, 0.0001, 0.0001, 0.0001, 0.000001] 
UB = [2000, 5.0, 5.0, 5.0, 1]

MyBounds = []
i = 0 
for i in range(0,len(LB)):
    MyBounds.append((LB[i], UB[i]))
    i = i+1

# Run Non-Linear Optimization Process
result = minimize(optimize1, initial_guess, bounds = MyBounds, args = (Size, Volatility, POV, Cost))
result


      fun: 4945541184.500702
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([     0.        ,  15830.99366068, -17642.97476486, -14305.11467421,
       -14877.31926118])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 288
      nit: 36
     njev: 48
   status: 0
  success: True
        x: array([8.82858409e+02, 3.53991453e-01, 7.55539680e-01, 8.24829592e-01,
       9.63999967e-01])

In [7]:
# Step 7. Calculate Non-Linear Regression Statistics

# Parameters
a1 = result.x.tolist()[0]
a2 = result.x.tolist()[1]
a3 = result.x.tolist()[2]
a4 = result.x.tolist()[3]
b1 = result.x.tolist()[4]


# ----- Calculate Estimated Cost
EstCost=(a1*Size**a2*Volatility**a3)*(b1*POV**a4+(1-b1))

#EstCost=EstCost.reshape(-1,1)

# Get Actual Cost
ActualCost=Cost
#ActualCost=MIData.iloc[:,4]

## Calculate Error
AvgCost=st.mean(ActualCost)
Error = ActualCost - EstCost
#MSE=1/(N-3-1)*sum((ActualCost-EstCost)**2)
MSE=1/N*sum((ActualCost-EstCost)**2)
SE=MSE**0.5

## Calculate the R2 of the MI Model
RSS=sum((ActualCost-EstCost)*(ActualCost-EstCost))
TSS=sum((ActualCost-AvgCost)*(ActualCost-AvgCost))
R2=1-RSS/TSS

# display results
a1, a2, a3, a4, b1, MSE, SE, R2

MiResults = pd.DataFrame([a1, a2, a3, a4, b1, MSE, SE, R2]).transpose()
MiResults.columns = ['a1', 'a2', 'a3', 'a4', 'b1', 'MSE', 'SE', 'R2']



In [8]:
# Step 8 - Show Results
MiResults

,a1,a2,a3,a4,b1,MSE,SE,R2
0,882.858409,0.353991,0.75554,0.82483,0.964,21213.207732,145.647546,0.124574


In [9]:
# Step 9. Save Results
MiResults.to_csv(OutputPath+MI_OutputDataFile, index = False, header = True)
#MiResults.transpose().to_csv(OutputPath+MI_OutputDataFile_T, index = True, header = False)